# Analyse des Équipements Sportifs dans le Val-d'Oise


Le présent notebook propose une analyse détaillée des équipements sportifs dans le département du Val-d'Oise. Les données utilisées proviennent du site data.iledefrance.fr, qui fournit un recensement exhaustif de ces infrastructures. L'URL source est : http://data.iledefrance.fr/explore/dataset/recensement_des_equipements_sportifs_dans_le_val-d_oise/download?format=csv. 
Ce jeu de données est essentiel pour comprendre la répartition géographique et la typologie des équipements sportifs dans la région.


L'objectif de cette analyse est de calculer le nombre d'équipements par commune et de fournir des détails nécessaires pour une carte et un affichage dynamique. Cela permettra d'identifier les zones les plus équipées et celles qui nécessitent un renforcement des infrastructures sportives.


## Méthodologie


La méthodologie utilisée dans ce notebook consiste en une analyse DuckDB des données, suivie d'une visualisation interactive avec Plotly Express. 
Tout d'abord, une requête SQL agrège les données par commune pour obtenir le nombre total d'équipements. 
Ensuite, une jointure avec les détails des équipements permet d'obtenir les informations nécessaires pour la visualisation. 
Enfin, les données sont converties en un DataFrame Pandas et utilisées pour créer une carte interactive représentant la répartition des équipements sportifs par commune.



## Analyse et Visualisation


Après avoir chargé et traité les données, nous procédons à la visualisation. 
La carte générée permet de visualiser la densité des équipements sportifs dans le Val-d'Oise, avec une échelle de couleur reflétant le nombre d'équipements par commune. 
Le survol d'une commune affiche le nom de la commune, son code INSEE, et le nombre d'équipements.


La mise en œuvre de cette analyse permet une compréhension plus fine de la distribution des infrastructures sportives dans le département et ouvre des perspectives pour des analyses plus approfondies sur l'accessibilité et la typologie de ces équipements.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("http://data.iledefrance.fr/explore/dataset/recensement_des_equipements_sportifs_dans_le_val-d_oise/download?format=csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH commune_equipements AS (
  SELECT 
    "new_name" AS nom_commune, 
    "new_code" AS code_insee, 
    COUNT(*) AS total_equipements
  FROM 
    loaded_dataset
  GROUP BY 
    "new_name", "new_code"
),
equipements_details AS (
  SELECT 
    "new_name" AS nom_commune, 
    "new_code" AS code_insee, 
    "equip_nom" AS equipement, 
    CAST("latitude" AS DOUBLE) AS lat, 
    CAST("longitude" AS DOUBLE) AS lon
  FROM 
    loaded_dataset
)
SELECT 
  ce.nom_commune, 
  ce.code_insee, 
  ce.total_equipements, 
  ed.equipement, 
  ed.lat, 
  ed.lon
FROM 
  commune_equipements ce
  LEFT JOIN equipements_details ed ON ce.code_insee = ed.code_insee
ORDER BY 
  ce.nom_commune """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est particulièrement adaptée pour créer des visualisations interactives et géospatiales comme des cartes. Plotly permet de représenter les données de manière interactive, facilitant l'exploration et la compréhension des informations spatiales. Cela est particulièrement utile pour visualiser la répartition d'équipements sportifs par commune.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px
from plotly.graph_objects import Figure
from plotly.subplots import make_subplots
import numpy as np

# Conversion en DataFrame agrégé (une ligne par commune)
df_communes = df.groupby(['nom_commune', 'code_insee', 'lat', 'lon']).agg({
    'total_equipements': 'first',
    'equipement': lambda x: '<br>'.join(x.astype(str))
}).reset_index()

# Ordre décroissant par nombre d'équipements
df_communes = df_communes.sort_values('total_equipements', ascending=False)

# Couleur basée sur le nombre d'équipements (bleu sport)
df_communes['color'] = pd.cut(df_communes['total_equipements'], 
                              bins=[0, 10, 50, 100, 200, 210], 
                              labels=['#E3F2FD', '#90CAF9', '#42A5F5', '#1E88E5', '#0D47A1'])

# Création de la carte
dataviz = px.scatter_mapbox(
    df_communes,
    lat='lat',
    lon='lon',
    color='total_equipements',
    size='total_equipements',
    hover_name='nom_commune',
    hover_data={
        'code_insee': True,
        'total_equipements': ':.0f',
        'equipement': False
    },
    color_continuous_scale=['#E3F2FD', '#64B5F6', '#2196F3', '#1565C0', '#0D47A1'],
    size_max=25,
    zoom=9,
    height=600,
    custom_data=['nom_commune']
)
dataviz.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>' +
                  'Code INSEE: %{customdata[1]}<br>' +
                  'Équipements: %{marker.size:.0f}<br><br>' +
                  '<i>Cliquez pour zoomer</i>'
)
dataviz.update_layout(
    mapbox_style="carto-positron",
    coloraxis_colorbar=dict(
        title='Équipements<br>            ',
        tickmode='linear',
        tick0=0,
        dtick=50
    ),
    margin=dict(l=20, r=20, t=40, b=20),
    title=dict(
        text='Équipements sportifs par commune',
        x=0.5,
        font=dict(size=20, family="Arial Black")
    )
)
dataviz.update_mapboxes(
    center={"lat": 48.95, "lon": 2.25},
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*